In [15]:
from pyomo.environ import *
from power import *
import numpy as np
import pandas as pd

In [16]:
#14bus
net = Network()

buses = [                                                 
    Bus(net, id= 1, bus_type='Slack'),
    Bus(net, id= 2, bus_type=   'PQ')
]

loads = [
    Load(id= 1, bus=buses[ 0], pb=100, p_input=21.70, q_input=12.70),
    Load(id= 2, bus=buses[ 1], pb=100, p_input=94.20, q_input=19.00)
]

generators = [
    Generator(id= 1, bus=buses[ 0], pb=100, p_max_input=5000, q_max_input=50, cost_a_input=1),
    Generator(id= 2, bus=buses[ 1], pb=100, p_max_input=200, q_max_input=200, cost_a_input=2),
]

lines = [
    Line(id= 1, from_bus=buses[ 0], to_bus=buses[ 1], r=0.01938, x=0.05917, b_half=0.0264, flow_max=99),     
    Line(id= 2, from_bus=buses[ 1], to_bus=buses[ 0], r=0.01938, x=0.05917, b_half=0.0264, flow_max=99),        
]

In [ ]:
class AC_OPF_IINJECTION:
    def __init__(self, network: Network):
        self.net = network

        # Generators, Loads, Buses, Lines
        self.generators = self.net.generators
        self.loads = self.net.loads
        self.buses = self.net.buses
        self.lines = self.net.lines

        # Pyomo Model
        self.model = ConcreteModel()
        self.model.name = 'AC OPF: Injeção de Corrente (Retangular)'

        # Constrói o modelo passo a passo
        self._create_sets()
        self._create_parameters()
        self._create_variables()
        self._create_constraints()
        self._create_objective()

        # Duais (se precisar extrair preços sombra)
        self.model.dual = Suffix(direction=Suffix.IMPORT)
    
    def _create_sets(self):    
        self.model.generators = Set(initialize=[g.name for g in self.generators], doc="Conjunto de geradores") # Cria conjunto de geradores com os nomes
        self.model.loads = Set(initialize=[l.name for l in self.loads], doc="Conjunto de cargas") # Cria conjunto de cargas
        self.model.buses = Set(initialize=[b.name for b in self.buses], doc="Conjunto de barras")  # Cria conjunto de barras
        self.model.lines = Set(initialize=[ln.name for ln in self.lines], doc="Conjunto de linhas") # Cria conjunto de linhas


    def _create_parameters(self):
        #Generators
        m = self.model
        m.generator_bus = Param(m.generators, initialize={g.name: g.bus.name for g in self.generators})
        m.generator_pmax = Param(m.generators, initialize={g.name: g.p_max for g in self.generators})
        m.generator_pmin = Param(m.generators, initialize={g.name: g.p_min for g in self.generators})
        m.generator_qmax = Param(m.generators, initialize={g.name: g.q_max for g in self.generators})
        m.generator_qmin = Param(m.generators, initialize={g.name: g.q_min for g in self.generators})
        m.generator_cost_a = Param(m.generators, initialize={g.name: g.cost_a for g in self.generators})
        m.generator_cost_b = Param(m.generators, initialize={g.name: g.cost_b for g in self.generators})
        m.generator_cost_c = Param(m.generators, initialize={g.name: g.cost_c for g in self.generators})

        #Loads
        m.load_bus = Param(m.loads, initialize={l.name: l.bus.name for l in self.loads})
        m.load_p= Param(m.loads, initialize={l.name: l.p for l in self.loads})
        m.load_q = Param(m.loads, initialize={l.name: l.q for l in self.loads})

        #Lines
        m.line_from = Param(m.lines, initialize={ln.name: ln.from_bus.name for ln in self.lines})
        m.line_to = Param(m.lines, initialize={ln.name: ln.to_bus.name for ln in self.lines})
        g_bus = self.net.get_G()
        b_bus = self.net.get_B()
        bus_names = [b.name for b in self.buses]
        g_dict = {(bus_names[i], bus_names[j]): g_bus[i, j] for i in range(len(bus_names)) for j in range(len(bus_names))}
        b_dict = {(bus_names[i], bus_names[j]): b_bus[i, j] for i in range(len(bus_names)) for j in range(len(bus_names))}

        m.G = Param(m.buses, m.buses, initialize=g_dict, doc="Real part of the admittance matrix")
        m.B = Param(m.buses, m.buses, initialize=b_dict, doc="Imaginary part of the admittance matrix")
        m.flow_max = Param(m.lines, initialize={ln.name: ln.flow_max_pu for ln in self.lines})

        #Bus
        m.bus_type = Param(m.buses, initialize={b.name: b.bus_type for b in self.buses})

    def _create_variables(self):
        m = self.model

        #Generators
        m.p = Var(m.generators, domain=NonNegativeReals, doc="Active Power Generation (pu)")
        m.q = Var(m.generators, domain=Reals, doc="Reactive Power Generation (pu)")

        #Buses
        m.v_r = Var(m.buses, domain=NonNegativeReals, doc="Bus Voltage Magnitude (pu)")
        m.v_i = Var(m.buses, domain=Reals, doc="Bus Voltage Angle (rad)")

        #Lines
        m.i_r = Var(m.lines, domain=Reals, doc="Line Current Magnitude (pu)")
        m.i_i = Var(m.lines, domain=Reals, doc="Line Current Imaginary (pu)")

    def _create_constraints(self):
        m = self.model

        #Generators
        slack_bus = next(b for b in m.buses if m.bus_type[b] == 'Slack') # Bus type = Slack - Fixing theta = 0
        m.v_r[slack_bus].fix(1)
        m.v_i[slack_bus].fix(0)

        def p_max_rule(m, g):
            return m.p[g] <= m.generator_pmax[g]
        m.p_max = Constraint(m.generators, rule=p_max_rule, doc="Active Power Generation Max")

        def p_min_rule(m, g):
            return m.p[g] >= m.generator_pmin[g]
        m.p_min = Constraint(m.generators, rule=p_min_rule, doc="Active Power Generation Min")

        def q_max_rule(m, g):
            return m.q[g] <= m.generator_qmax[g]
        m.q_max = Constraint(m.generators, rule=q_max_rule, doc="Reactive Power Generation Max")

        def q_min_rule(m, g):
            return m.q[g] >= m.generator_qmin[g]
        m.q_min = Constraint(m.generators, rule=q_min_rule, doc="Reactive Power Generation Min")

        #Buses
        def v_mag_rule(m, b):
            return inequality(0.95**2, m.v_r[b]**2 + m.v_i[b]**2, 1.05**2)
        m.v_max = Constraint(m.buses, rule=v_mag_rule, doc="Bus Voltage Magnitude Max")

        #Lines
        def real_current_ruke()







        def flow_out_rule(m, ln):
            i = m.line_from[ln]
            j = m.line_to[ln]
            flow = m.v[i]**2 * m.G[i, j] - m.v[i] * m.v[j] * (m.G[i, j] * cos(m.theta[i] - m.theta[j]) + m.B[i, j] * sin(m.theta[i] - m.theta[j]))
            return m.p_flow_out[ln] == flow
        m.flow_out_rule = Constraint(m.lines, rule=flow_out_rule, doc="Flow Out Rule")

        def flow_out_bounds(m, ln):
            return inequality(-m.flow_max[ln], m.p_flow_out[ln], m.flow_max[ln])
        m.flow_out_bounds = Constraint(m.lines, rule=flow_out_bounds, doc="Flow Out Bounds")

        def flow_in_rule(m, ln):
            i = m.line_from[ln]
            j = m.line_to[ln]
            flow = m.v[j]**2 * m.G[j, i] - m.v[j] * m.v[i] * (m.G[j, i] * cos(m.theta[j] - m.theta[i]) + m.B[j, i] * sin(m.theta[j] - m.theta[i]))
            return m.p_flow_in[ln] == flow
        m.flow_in_rule = Constraint(m.lines, rule=flow_in_rule, doc="Flow In Rule")

        def flow_in_bounds(m, ln):
            return inequality(-m.flow_max[ln], m.p_flow_in[ln], m.flow_max[ln])
        m.flow_in_bounds = Constraint(m.lines, rule=flow_in_bounds, doc="Flow In Bounds")

        def flow_out_bounds_q(m, ln):
            i = m.line_from[ln]
            j = m.line_to[ln]
            flow = m.v[i]**2 * m.B[i, j] - m.v[i] * m.v[j] * (m.G[i, j] * sin(m.theta[i] - m.theta[j]) - m.B[i, j] * cos(m.theta[i] - m.theta[j]))
            return m.q_flow_out[ln] == flow
        m.flow_out_bounds_q = Constraint(m.lines, rule=flow_out_bounds_q, doc="Flow Out Bounds Q")

        def flow_in_bounds_q(m, ln):
            i = m.line_from[ln]
            j = m.line_to[ln]
            flow = m.v[j]**2 * m.B[j, i] - m.v[j] * m.v[i] * (m.G[j, i] * sin(m.theta[j] - m.theta[i]) - m.B[j, i] * cos(m.theta[j] - m.theta[i]))
            return m.q_flow_in[ln] == flow
        m.flow_in_bounds_q = Constraint(m.lines, rule=flow_in_bounds_q, doc="Flow In Bounds Q")

        #Power Balance
        def active_power_balance_rule(m, bus):
            active_generation = sum(m.p[g] if m.generator_bus[g] == bus else 0 for g in m.generators)
            active_load = sum(m.load_p[l] if m.load_bus[l] == bus else 0 for l in m.loads)
            active_flow_out = sum(m.p_flow_out[ln] if  m.line_from[ln] == bus else 0 for ln in m.lines)
            active_flow_in = sum(m.p_flow_in[ln] if m.line_to[ln] == bus else 0 for ln in m.lines)
            return active_generation + active_flow_in - active_flow_out == active_load
        m.active_power_balance = Constraint(m.buses, rule=active_power_balance_rule, doc="Active Power Balance")

        def reactive_power_balance_rule(m, bus):
            reactive_generation = sum(m.q[g] if m.generator_bus[g] == bus else 0 for g in m.generators)
            reactive_load = sum(m.load_q[l] if m.load_bus[l] == bus else 0 for l in m.loads)
            reactive_flow_out = sum(m.q_flow_out[ln] if m.line_from[ln] == bus else 0 for ln in m.lines)
            reactive_flow_in = sum(m.q_flow_in[ln] if m.line_to[ln] == bus else 0 for ln in m.lines)
            return reactive_generation + reactive_flow_in - reactive_flow_out == reactive_load
        m.reactive_power_balance = Constraint(m.buses, rule=reactive_power_balance_rule, doc="Reactive Power Balance")

    def _create_objective(self):
        m = self.model
        def objective_rule(m):
            cost = sum(m.generator_cost_a[g] * m.p[g] for g in m.generators)
            return cost
        m.objective = Objective(rule=objective_rule, sense=minimize, doc="Objective Function")

    
    def _create_results(self):
        m = self.model
        self.results = {}
        self.results['objective'] = value(m.objective)
        self.results['p'] = {g: value(m.p[g]) for g in m.generators}
        self.results['q'] = {g: value(m.q[g]) for g in m.generators}
        self.results['v'] = {b: value(m.v[b]) for b in m.buses}
        self.results['theta'] = {b: value(m.theta[b]) for b in m.buses}
        self.results['p_flow_out'] = {ln: value(m.p_flow_out[ln]) for ln in m.lines}
        self.results['p_flow_in'] = {ln: value(m.p_flow_in[ln]) for ln in m.lines}
        self.results['q_flow_out'] = {ln: value(m.q_flow_out[ln]) for ln in m.lines}
        self.results['q_flow_in'] = {ln: value(m.q_flow_in[ln]) for ln in m.lines}
    
    def solve(self, solver_name='ipopt', tee=False):
        solver = SolverFactory('ipopt', teee=tee)
        solver.solve(self.model)
        self._create_results()
        return self.results

In [18]:
dispatch_model = Economic_Dispatch(net)

In [19]:
dispatch_model.model.pprint()

4 Set Declarations
    buses : Conjunto de barras
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'Bus 1', 'Bus 2'}
    generators : Conjunto de geradores
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'Generator 1', 'Generator 2'}
    lines : Conjunto de linhas
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'Line 1', 'Line 2'}
    loads : Conjunto de cargas
        Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'Load 1', 'Load 2'}

17 Param Declarations
    B : Size=4, Index=buses*buses, Domain=Any, Default=None, Mutable=False
        Key                : Value
        ('Bus 1', 'Bus 1') : -30.473373046359104
        ('Bus 1', 'Bus 2') :  30.52617304635910

In [22]:
results = dispatch_model.solve(tee=True)


In [21]:
results = pd.DataFrame(results)
results

,objective,p,q,v,theta,p_flow_out,p_flow_in,q_flow_out,q_flow_in
Generator 1,210.1,0.217,0.127,NaN,NaN,NaN,NaN,NaN,NaN
Generator 2,210.1,0.942,0.190,NaN,NaN,NaN,NaN,NaN,NaN
Bus 1,210.1,NaN,NaN,1.0000,0.000000,NaN,NaN,NaN,NaN
Bus 2,210.1,NaN,NaN,1.0015,-0.007558,NaN,NaN,NaN,NaN
Line 1,210.1,NaN,NaN,NaN,NaN,-0.216345,0.215750,61.172937,61.113222
Line 2,210.1,NaN,NaN,NaN,NaN,0.215750,-0.216345,61.113222,61.172937
